In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from itertools import cycle


from multiprocessing import Pool, cpu_count
print(f'Parallelism on {cpu_count()} CPU')

Parallelism on 8 CPU


In [2]:
# Read in the data
INPUT_DIR = './m5-forecasting-accuracy'
df_sale = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
df_calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
df_price = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')

In [3]:

columns = df_sale.columns
date_columns = columns[columns.str.contains("d_")]
dates_s = [
    pd.to_datetime(df_calendar.loc[df_calendar['d'] == str_date,'date'].values[0]) 
           for str_date in date_columns
]

tmp = df_sale[date_columns].sum()
ignore_date = df_calendar[df_calendar['d'].isin(tmp[tmp < 10000].index.values)]['date'].values

df_ev_1 = pd.DataFrame({'holiday': 'Event 1', 'ds': df_calendar[~df_calendar['event_name_1'].isna()]['date']})
df_ev_2 = pd.DataFrame({'holiday': 'Event 2', 'ds': df_calendar[~df_calendar['event_name_2'].isna()]['date']})
df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': df_calendar[df_calendar['snap_CA'] == 1]['date']})
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': df_calendar[df_calendar['snap_TX'] == 1]['date']})
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': df_calendar[df_calendar['snap_WI'] == 1]['date']})
holidays = pd.concat((df_ev_1, df_ev_2, df_ev_3, df_ev_4, df_ev_5))


In [4]:
#forecast with prophet without holiday
ids1 = np.array(['FOODS_3_090_WI_3_validation','FOODS_3_785_CA_1_validation'])
#forecast with prophet with holiday
ids2 = np.array(['FOODS_3_362_CA_3_validation'])
#forecast with prophet top down
ids3 = df_sale['id'].values
#forecast with ma
ids4 = np.array(['FOODS_3_090_CA_3_validation'])

#other predict
ids = np.concatenate((ids1, ids2, ids4), axis=0)
#Default prophet top down
ids3 = ids3[~np.isin(ids3,ids)]


print(f'Prophet without: {ids1.shape[0]}, Prophet bu: {ids2.shape[0]}, Prophet td: {ids3.shape[0]}, MA : {ids4.shape[0]}')
print(f'Total ids: {ids1.shape[0] + ids2.shape[0] + ids3.shape[0] + ids4.shape[0]} , Total sample: {df_sale.shape[0]}')


Prophet without: 2, Prophet bu: 1, Prophet td: 30486, MA : 1
Total ids: 30490 , Total sample: 30490


### MP runner

In [5]:
import util

In [6]:
def mp_runner(func, paraMap):
    with Pool(cpu_count()) as p:
        predictions  = list(p.starmap(func, paraMap))   
    return predictions

In [7]:
# %%time
    
# pred = mp_runner(util.run_prophet_with_holidays_infomation, [(id, df_sale, dates_s, date_columns, holidays) for id in ids1])

# pred[0]['yhat'].head()

# Submission

In [7]:

ss = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

In [8]:
ss.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_sale.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [10]:

ss_new = pd.DataFrame(columns=ss.columns)

In [11]:
ss_new

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28


In [32]:
target_id = list()
for uni_cat in df_sale['cat_id'].unique():
    idx_with_same_cat_id = df_sale.loc[uni_cat == df_sale["cat_id"]]["id"]
    target_id.append(idx_with_same_cat_id.iloc[0])
target_id

['HOBBIES_1_001_CA_1_validation',
 'HOUSEHOLD_1_001_CA_1_validation',
 'FOODS_1_001_CA_1_validation']

In [35]:
%%time
chunk = 1
history_window = len(date_columns)

idx_with_param = [
    (i, df_sale, dates_s, date_columns, holidays, history_window) 
        for i in target_id
]
predictions = mp_runner(util.run_prophet_with_holidays_infomation, idx_with_param)



CPU times: user 1.01 s, sys: 1.76 s, total: 2.77 s
Wall time: 16.1 s


In [47]:
predictions[0]['yhat']

0     0.836813
1     0.895571
2     0.780819
3     0.772262
4     0.823407
5     0.941348
6     0.822398
7     0.779004
8     0.836396
9     0.748963
10    0.679992
11    0.760205
12    0.935479
13    0.751099
14    0.734159
15    0.819379
16    0.668205
17    0.657899
18    0.735911
19    0.855440
20    0.730263
21    0.713258
22    0.772086
23    0.658119
24    0.651176
25    0.704867
26    0.856873
27    0.737001
Name: yhat, dtype: float64

In [49]:
target_id[0], target_id[1], target_id[2]

('HOBBIES_1_001_CA_1_validation',
 'HOUSEHOLD_1_001_CA_1_validation',
 'FOODS_1_001_CA_1_validation')

In [52]:

pred_startswith_item_name = [
    [target_id[p_id]] + list(predictions[p_id]['yhat']) 
        for p_id in range(len(predictions))
]
pred_df = pd.DataFrame(pred_startswith_item_name, columns=ss.columns)
pred_df.to_csv(f"submission_prophet.csv")

In [53]:
pred_df.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.836813,0.895571,0.780819,0.772262,0.823407,0.941348,0.822398,0.779004,0.836396,...,0.735911,0.855440,0.730263,0.713258,0.772086,0.658119,0.651176,0.704867,0.856873,0.737001
1,HOUSEHOLD_1_001_CA_1_validation,0.656762,0.545865,0.668083,0.630089,0.769636,0.910281,0.742247,0.740738,0.602411,...,0.869845,0.993813,0.802282,0.834135,0.707514,0.812353,0.755707,0.875735,0.984410,0.817227
2,FOODS_1_001_CA_1_validation,0.765004,0.705642,0.797636,0.732782,1.006793,0.968723,0.936054,0.904861,0.795251,...,1.016622,1.189292,1.096173,0.847757,0.783213,0.869216,0.797864,1.065174,1.140959,1.101959
